In [1]:
%pwd


'x:\\CampusX\\projects\\movie_recommendation_project\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'x:\\CampusX\\projects\\movie_recommendation_project'

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.movie_recommendation_project.constants import *
from src.movie_recommendation_project.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )
        return data_ingestion_config



In [9]:
import os
import urllib.request as request
import zipfile
from src.movie_recommendation_project.logging import logger
from src.movie_recommendation_project.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig) :
        self.config=config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers= request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )

            logger.info(f"{filename} download with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size : {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)


In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-30 06:54:30,004: INFO :common :yaml file: config\config.yaml loaded successfully]
[2024-08-30 06:54:30,019: INFO :common :yaml file: params.yaml loaded successfully]
[2024-08-30 06:54:30,022: INFO :common :created directory at: artifacts]
[2024-08-30 06:54:30,025: INFO :common :created directory at: artifacts/data_ingestion]
[2024-08-30 06:54:31,855: INFO :2700526799 :artifacts/data_ingestion/top_10000_movies_data.zip download with following info: 
Connection: close
Content-Length: 2075640
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "64144c0df22720064cb67dc6debca728f727beadbabe0d02a7d8e43eb99523e5"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: EA68:3E53D5:5C7B3:7E0A8:66D11F50
Accept-Ranges: bytes
Date: Fri, 30 Aug 2024 01:24:32 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4724-B